In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
# from backtesting.portfolio_cache import load_cached_object,cache_backtest

In [4]:
from backtesting.backtest import Backtest
from backtesting.scenarios import scenario_no_capital_growth, scenario_with_capital_growth

In [27]:
backtest_single = Backtest.from_scenario(scenario_no_capital_growth)
backtest_single.run()


Backtest starting... whomp whomp!
Total trading days: 82
Universe size: 480
Total strategies: 1
Starting in 2025-02-01
Ending in 2025-06-01


Backtesting by date x strategy: 100%|██████████| 82/82 [00:00<00:00, 143.66day/s]

Ding ding ding! Backtest completed!


In [28]:
backtest_batch = Backtest.from_scenario(scenario_with_capital_growth)
backtest_batch.run_batch()

Backtest starting... swoosh!
Universe size: 480
Total trading days: 82
Total strategies: 1
Starting in 2025-02-01
Ending in 2025-06-01


Backtesting by strategy: 100%|██████████| 1/1 [00:00<00:00,  2.72strategy/s]

Backtest completed!


In [29]:
asingle = backtest_single.generate_analytics()
# ab = backtest_batch.generate_analytics()

In [30]:
from report import generate_simple_report

generate_simple_report(asingle, "2022-01-01", "2025-06-01")
# generate_simple_report(ab, "2022-01-01", "2025-06-01")


Report saved to: reports/portfolio_report_20250610_002234.pdf


'reports/portfolio_report_20250610_002234.pdf'

In [ ]:
from config import Strategies
from strategies.strategy import Strategy


universe = backtest_single.portfolio.get_universe()
strategies = [Strategy.create(s) for s in [Strategies.RSI_CROSSOVER]]
max_lookback = max(strategy.min_window for strategy in strategies)
data_start_date = pd.Timestamp(backtest_single.start_date) - pd.Timedelta(days=max_lookback)
signals = {}

count =0 
for date in backtest_single.trading_dates:

    for strategy in strategies:
        price_type = strategy.price_type
        prices = backtest_single.portfolio.get_prices(
            price_type, end_date=date, start_date=data_start_date # uses prev close price
        )
        signal = generate_signals_single_date(prices.loc[:, universe])
    signals[date] = signal


In [127]:
universe = backtest_batch.portfolio.get_universe()
strategies = [Strategy.create(s) for s in [Strategies.RSI_CROSSOVER]]
price_type = "close"
max_lookback = max(strategy.min_window for strategy in strategies)
data_start_date = pd.Timestamp(backtest_batch.start_date) - pd.Timedelta(days=max_lookback)
pricesb = backtest_batch.portfolio.get_prices(price_type, start_date=data_start_date, end_date=backtest_batch.end_date)[universe]

run_start_date = pricesb.loc[backtest_batch.start_date:, :].index[0] # start date may fall on a weekend, we find the closest biz date that has price data as run start date
data_r = pricesb.reset_index()
run_start_index = data_r[data_r["Date"] == run_start_date].index[0]
del data_r

signalsb = []
for strategy in strategies:
    signalb = generate_signals_batch(pricesb, run_start_index)
    signalsb.append(signalb)